In [193]:
from matplotlib.axes import Axes
import matplotlib.axis as maxis
from matplotlib.collections import LineCollection
from matplotlib.patches import Circle
from matplotlib.projections import register_projection
import matplotlib.spines as mspines
from matplotlib.ticker import MultipleLocator, NullFormatter, ScalarFormatter
import matplotlib.transforms as transforms
import matplotlib.gridspec as gridspec
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import Dropdown, Checkbox, interact, interactive, interact_manual, fixed, Select, FloatRangeSlider, FloatSlider, FloatText, Textarea

#from _util import colored_line
from metpy.calc import (dewpoint, dry_lapse, moist_lapse, vapor_pressure, saturation_vapor_pressure,
                        get_wind_speed, get_wind_dir, pressure_to_height_std, mixing_ratio, cape_cin, get_wind_components,
                        height_to_pressure_std, equivalent_potential_temperature, parcel_profile, precipitable_water,
                        storm_relative_helicity)
from metpy.calc.tools import delete_masked_points, log_interp, get_layer
from metpy.package_tools import Exporter
from metpy.units import units
from metpy.plots import SkewT
from metpy.plots import Hodograph
import metpy.calc as metcalc
import metpy.calc as mpcalc
from metpy.units import concatenate
from metpy.constants import Rd

import numpy.ma as ma
from netCDF4 import Dataset
import netCDF4
from netCDF4 import num2date, date2num
from datetime import datetime
import math

#sounding = np.genfromtxt("testsounding.txt", usecols = (0, 2, 3, 6, 7, 1))
from datetime import datetime
from metpy.io import get_upper_air_data

In [194]:
def bunkers_storm_motion(u, v, p, hgt):

    r"""Calculate the Bunkers RM and LM storm motions, as 
    well as the mean sfc-6km flow for ordinary cell motion.
    Uses the storm motion calculation from [Bunkers et al, 2000]_,
    which does not require effective inflow layer calculations,
    unlike the more recent version in [Bunkers et al 2014]_.
    
    Parameters
    ----------
    u : array-like
        U component of the wind
    v : array-like
        V component of the wind
    p : array-like
        Pressure from sounding
    hgt : array-like
        Heights from sounding
    
    Returns
    -------
    RM_vector
        U and v component of Bunkers RM storm motion
    LM_vector
        U and v component of Bunkers LM storm motion
    mean_vector
        U and v component of sfc-6km mean flow
        
    See Also
    --------
    bunkers_storm_motion_2014
    
    Citations:
    
    Bunkers, M. J., B. A. Klimowski, J. W. Zeitler, R. L. Thompson, and M. L. Weisman, 2000:
        Predicting supercell motion using a new hodograph technique. Wea. Forecasting, 15, 61–79.
    
    
    Bunkers, M. J., D. A. Barber, R. L. Thompson, R. Edwards, and J. Garner, 2014: 
        Choosing a universal mean wind for supercell motion prediction. J. Operational Meteor.,
        2 (11), 115–129, doi: https://dx.doi.org/10.15191/nwajom.2014.0211.
    
    
     """

    u = u.to('m/s')
    v = v.to('m/s')
    
    
    #mean wind from sfc-6km
    #correct height to be height AGL
    u6 = mean_wind_pressure_weighted(u, v, p, hgt, depth = 6000 * units('meter'))
    u6m = u6[0]
    v6m = u6[1]

    #mean wind from sfc-500m

    u5 = mean_wind_pressure_weighted(u, v, p, hgt, depth = 500 * units('meter'))
    u5m = u5[0]
    v5m = u5[1]

    #mean wind from 5.5-6km

    u55 = mean_wind_pressure_weighted(u, v, p, hgt, depth = 500 * units('meter'), bottom = 5500 * units('meter'))
    u55m = u55[0]
    v55m = u55[1]

    #Calculate the shear vector from sfc-500m to 5.5-6km

    u6shr = u55m - u5m
    v6shr = v55m - v5m

    #making the shear vector

    vl = [u6shr.magnitude, v6shr.magnitude, 0]

    #Create a k hat vector

    vk = [0,0,1]

    #Take the cross product of the wind shear and k, and divide by the vector magnitude and multiply by D (7.5 m/s or 14.5788 kt)
    rdev = np.cross(vl, vk)*(7.5/(np.sqrt(u6shr.magnitude ** 2 + v6shr.magnitude ** 2)))
    ldev = np.cross(vk, vl)*(7.5/(np.sqrt(u6shr.magnitude ** 2 + v6shr.magnitude ** 2)))
    
    #Add the deviations to the layer average wind to get the RM motion
    uRM = u6m.magnitude + rdev[0]
    vRM = v6m.magnitude + rdev[1]

    #Subtract the deviations to get the LM motion
    uLM = u6m.magnitude - rdev[0]
    vLM = v6m.magnitude - rdev[1]
    
    RM_vector = np.asarray([uRM, vRM])*units('m/s')
    
    LM_vector = np.asarray([uLM, vLM])*units('m/s')
    
    mean_vector = np.asarray([u6m.magnitude, v6m.magnitude])*units('m/s')
    
    return RM_vector, LM_vector, mean_vector


In [195]:
def most_unstable_parcel(pressure, temperature, dewpoint, heights=None,
                         bottom=None, depth=300 * units.hPa):
    """
    Determine the most unstable parcel in a layer.
    Determines the most unstable parcle of air by calculating the equivalent
    potential temperature and finding its maximum in the specified layer.
    Parameters
    ----------
    pressure: `pint.Quantity`
        Atmospheric pressure profile
    temperature: `pint.Quantity`
        Atmospheric temperature profile
    dewpoint: `pint.Quantity`
        Atmospheric dewpoint profile
    heights: `pint.Quantity`, optional
        Atmospheric height profile. Standard atmosphere assumed when None (the default).
    bottom: `pint.Quantity`, optional
        Bottom of the layer to consider for the calculation in pressure or height.
        Defaults to using the bottom pressure or height.
    depth: `pint.Quantity`, optional
        Depth of the layer to consider for the calculation in pressure or height. Defaults
        to 300 hPa.
    Returns
    -------
    `pint.Quantity`
        Pressure, temperature, and dew point of most unstable parcel in the profile.
    See Also
    --------
    get_layer
    """
    p_layer, T_layer, Td_layer = get_layer(pressure, temperature, dewpoint, bottom=bottom,
                                           depth=depth, heights=heights, interpolate=False)
    theta_e = equivalent_potential_temperature(p_layer, T_layer, Td_layer)
    max_idx = np.argmax(theta_e)
    return p_layer[max_idx], T_layer[max_idx], Td_layer[max_idx], max_idx

In [196]:
def mucape_cin(pressure, temperature, dewpoint, **kwargs):
    r"""Calculate MUCAPE and MUCIN.
    Calculate the convective available potential energy (CAPE) and convective inhibition (CIN)
    of a given upper air profile and most unstable parcel path. CIN is integrated between the
    surface and LFC, CAPE is integrated between the LFC and EL (or top of sounding).
    Intersection points of the measured temperature profile and parcel profile are linearly
    interpolated.
    Parameters
    ----------
    pressure : `pint.Quantity`
        The atmospheric pressure level(s) of interest. The first entry should be the starting
        point pressure.
    temperature : `pint.Quantity`
        The sounding temperature
    dewpoint : `pint.Quantity`
        The sounding dew point
    Returns
    -------
    `pint.Quantity`
        Convective available potential energy (CAPE) for the most unstable parcel.
    `pint.Quantity`
        Convective inhibition (CIN) for the most unstable parcel.
    See Also
    --------
    cape_cin, lfc, el
    """
    mu_parcel = most_unstable_parcel(pressure, temperature, dewpoint, **kwargs)
    max_idx = mu_parcel[3]
    mu_pressure_profile = pressure[max_idx:]
    mu_temperature_profile = temperature[max_idx:]
    mu_dewpoint_profile = dewpoint[max_idx:]
    mu_profile = parcel_profile(mu_pressure_profile, mu_parcel[1], mu_parcel[2])
    return cape_cin(mu_pressure_profile, mu_temperature_profile,
                    mu_dewpoint_profile, mu_profile)

In [197]:
def bulk_shear(u, v, pressure, height=None, bottom=None, depth=None):
    r"""Calculate bulk shear through a layer. 
    
    Layer top and bottom specified in meters or pressure.
    
    Parameters
    ----------
    u : `pint.Quantity`
        U-component of wind.
    v : `pint.Quantity`
        V-component of wind.
    p : `pint.Quantity`
        Atmospheric pressure profile
    height : `pint.Quantity`
        Heights from sounding
    depth: `pint.Quantity`
        The depth of the layer in meters or hPa
    bottom: `pint.Quantity`
        The bottom of the layer in meters or hPa.
        If in meters, must be in the same coordinates as the given
        heights (i.e., don't use meters AGL unless given heights
        are in meters AGL.) Default is the surface (1st observation.)
        
    Returns
    -------
    `pint.Quantity'
        u_shr: u-component of layer bulk shear, in m/s
    `pint.Quantity'
        v_shr: v-component of layer bulk shear, in m/s
    `pint.Quantity'
        shr_mag: magnitude of layer bulk shear, in m/s
        
    """   
    
    
    u = u.to('meters/second')
    v = v.to('meters/second')
    
    sort_inds = np.argsort(pressure[::-1])
    pressure = pressure[sort_inds]
    height = height[sort_inds]
    u = u[sort_inds]
    v = v[sort_inds]
        
    w_int = get_layer(pressure, u, v, heights=height, bottom=bottom, depth=depth)
    
    u_shr = w_int[1][-1] - w_int[1][0]
    v_shr = w_int[2][-1] - w_int[2][0]
    
    shr_mag = np.sqrt((u_shr ** 2) + (v_shr ** 2))
    
    return u_shr, v_shr, shr_mag

    

In [198]:
#Supercell composite calculation. Should use mucape, esrh, and ebwd, but for now we'll use sbcape, sfc-3 srh, and sfc-6 shear
def supercell_comp(mucape, esrh, ebwd):
    r"""Calculate the supercell composite parameter.
     
    The supercell composite parameter is designed to identify
    environments favorable for the development of supercells,
    and is calculated using the formula developed by 
    [Thompson, Edwards, and Mead, 2004]:
    
    SCP = (mucape / 1000 J/kg) * (esrh / 50 m^2/s^2) * (ebwd / 20 m/s) 
    
    The ebwd term is set to zero below 10 m/s and
    capped at 1 when ebwd exceeds 20 m/s.
    
    Parameters
    ----------
    mucape : array-like
        Most-unstable CAPE
    esrh : array-like
        Effective-layer storm-relative helicity
    ebwd : array-like
        Effective bulk shear
        
    Returns
    -------
    number
        supercell composite
        
    Citation:
    Thompson, R.L., R. Edwards, and C. M. Mead, 2004b:
        An update to the supercell composite
        and significant tornado parameters.
        Preprints, 22nd Conf. on Severe Local
        Storms, Hyannis, MA, Amer.
        Meteor. Soc.
        
    """
    
    ebwd = ebwd.to('m/s')
    ebwd = ebwd.magnitude
    inds = np.where((ebwd <= 20.) & (ebwd >= 10.))
    inds1 = np.where(ebwd < 10.)
    inds2 = np.where(ebwd > 20.)
    ebwd[inds] = ebwd[inds] / 20.
    ebwd[inds1] = 0.
    ebwd[inds2] = 1.
    sup_comp = (mucape.magnitude / 1000.) * (esrh.magnitude / 50.) * ebwd
     
    return sup_comp

In [199]:
def sigtor(sbcape, sblcl, srh1, shr6):
    r"""Calculate the significant tornado parameter (fixed layer).
     
    The significant tornado parameter is designed to identify
    environments favorable for the production of significant
    tornadoes contingent upon the development of supercells.
    It's calculated according to the formula used on the SPC
    mesoanalysis page, updated in [Thompson, Edwards, and Mead, 2004]:
    
    sigtor = (sbcape / 1500 J/kg) * ((2000 m - sblcl) / 1000 m) * (srh1 / 150 m^s/s^2) * (shr6 / 20 m/s)
    
    The sblcl term is set to zero when the lcl is above 2000m and 
    capped at 1 when below 1000m, and the shr6 term is set to 0 
    when shr6 is below 12.5 m/s and maxed out at 1.5 when shr6
    exceeds 30 m/s.
    
    Parameters
    ----------
    sbcape : array-like
        Surface-based CAPE
    sblcl : array-like
        Surface-based lifted condensation level
    srh1 : array-like
        Surface-1km storm-relative helicity
    shr6 : array-like
        Surface-6km bulk shear
        
    Returns
    -------
    number
        significant tornado parameter
    
    Citation:
    Thompson, R.L., R. Edwards, and C. M. Mead, 2004b:
        An update to the supercell composite
        and significant tornado parameters.
        Preprints, 22nd Conf. on Severe Local
        Storms, Hyannis, MA, Amer.
        Meteor. Soc.
        
    """
    
    shr6 = shr6.to('m/s')
    shr6 = shr6.magnitude
    sblcl = sblcl.magnitude
    ind = np.where((sblcl <= 2000.) & (sblcl >= 1000.))
    ind1 = np.where(sblcl < 1000.)
    ind2 = np.where(sblcl > 2000.)
    sind = np.where((shr6 <= 30.) & (shr6 >= 12.5))
    sind1 = np.where(shr6 < 12.5)
    sind2 = np.where(shr6 > 30.)
    sblcl[ind] = (2000. - sblcl[ind]) / 1000.
    sblcl[ind1] = 1.
    sblcl[ind2] = 0.
    shr6[sind] = shr6[sind] / 20.
    shr6[sind1] = 0.
    shr6[sind2] = 1.5
    sigtor = (sbcape.magnitude / 1500.) * sblcl * (srh1.magnitude / 150.) * shr6
     
    return sigtor

In [200]:
def mean_wind_pressure_weighted(u, v, p, hgt, depth, bottom=None, obs_only=False):
    r"""Calculate pressure-weighted mean wind through a layer. 
    
    Layer top and bottom specified in meters AGL.
    
    Parameters
    ----------
    u : array-like
        U-component of wind.
    v : array-like
        V-component of wind.
    p : array-like
        Atmospheric pressure profile
    hgt : array-like
        Heights from sounding
    depth: `pint.Quantity`
        The depth of the layer in meters.
    bottom: `pint.Quantity`
        The bottom of the layer in meters AGL.
        Default is the first observation, assumed to be the surface.
        
    Returns
    -------
    `pint.Quantity`
        u_mean: u-component of layer mean wind, in m/s
    `pint.Quantity`
        v_mean: v-component of layer mean wind, in m/s

    """  
    u = u.to('meters/second')
    v = v.to('meters/second')
    if bottom:
        bottom = bottom + hgt[0]
    w_int = get_layer(p, u, v, heights=hgt, bottom=bottom, depth=depth)
    
    if obs_only:
        u_mean = ma.average(w_int[1], weights = w_int[0]) * units('m/s')
        v_mean = ma.average(w_int[2], weights = w_int[0]) * units('m/s')
    
    else:
        u_mean = np.trapz(w_int[1] * w_int[0], x=w_int[0]) / np.trapz(w_int[0], x=w_int[0]) * units('m/s')
        v_mean = np.trapz(w_int[2] * w_int[0], x=w_int[0]) / np.trapz(w_int[0], x=w_int[0]) * units('m/s')
    
    return u_mean, v_mean

In [201]:
#%%timeit
#Let's try an effective layer calculation!
#truncate a layer at 500mb, since the effective inflow top should probably never be above there
#levc_t = get_layer(levc, levc, bottom = levc[0], depth=levc[0] - 500 * units('hPa'))
#capey = np.zeros((levc_t[0].shape[0]))
#ciny = np.zeros((levc_t[0].shape[0]))
#Find the bottom of the effective layer:
#cape_lim = 100.
#cin_lim = -250.
def effective_layer(p, T, Td, cape_lim = 100. * units('J/kg'), cin_lim = -250. * units('J/kg')):
    base_ind = np.nan
    top_ind = np.nan
    mucape, _ = mucape_mucin(p, T, Td)
    if mucape > 100 * units('J/kg'):
        for i in range(p.shape[0]):
            try:
                prof_c = metcalc.parcel_profile(p[i:], T[i], Td[i]).to('degC')
            except:
                continue
            #Feed it into the cape function to get cape and cin
            capearr = cape_cin(p[i:], T[i:], Td[i:], prof_c)
            #print(capearr)
            if ((capearr[0].magnitude >= cape_lim.magnitude) & (capearr[1].magnitude >= cin_lim.magnitude)):
                base_ind = i
                break
        if np.isnan(base_ind) == False:
            #Keep going up to find the effective inflow top
            for i in range(base_ind+1, p.shape[0]):
                try:
                    prof_c = metcalc.parcel_profile(p[i:], T[i], Td[i]).to('degC')
                except:
                    top_ind = i-1
                    break
                capearr = cape_cin(p[i:], T[i:], Td[i:], prof_c)
                #print(capearr)
                if ((capearr[0].magnitude < cape_lim.magnitude) or (capearr[1].magnitude < cin_lim.magnitude)):
                    top_ind = i-1
                    break
    return base_ind, top_ind

In [202]:
def critical_angle(u, v, pressure, height, stormu, stormv):
    r"""Calculate the critical angle.
    
    The critical angle is the angle between the 10m storm-relative inflow vector
    and the 10m-500m shear vector. A critical angle near 90 degrees indicates 
    that a storm in this environment on the indicated storm motion vector 
    is likely ingesting purely streamwise vorticity into its updraft, and Esterheld and 
    Giuliana (2008) showed that significantly tornadic supercells tend to occur in environments
    with critical angles near 90 degrees.

    Parameters
    ----------
    
    u : array-like
        U-component of sounding winds.
    v : array-like
        V-component of sounding winds.
    pressure : array-like
        Pressures from sounding.
    height : array-like
        Heights from sounding.
    stormu : array-like
        U-component of storm motion.
    stormv :
        V-component of storm motion.
        
    Returns
    -------
    `pint.Quantity'
        critical angle in degrees
        
    
    Esterheld, J., & Giuliano, D. (2008). Discriminating between Tornadic and Non-Tornadic Supercells:
    A New Hodograph Technique. E-Journal Of Severe Storms Meteorology, 3(2).Retrieved July 10, 2017, 
    from http://ejssm.org/ojs/index.php/ejssm/article/view/33/38""" 
    
    #Convert everything to m/s
    u = u.to('m/s')
    v = v.to('m/s')
    stormu = stormu.to('m/s')
    stormv = stormv.to('m/s')
    
    sort_inds = np.argsort(pressure[::-1])
    pressure = pressure[sort_inds]
    height = height[sort_inds]
    u = u[sort_inds]
    v = v[sort_inds]

    #Calculate sfc-500m shear vector
    shr5 = bulk_shear(u, v, pressure, height=height, depth = 500 * units('meter'))
    
    #Make everything relative to the sfc wind orientation
    umn = stormu - u[0]
    vmn = stormv - v[0]
    
    vshr = np.asarray([shr5[0].magnitude, shr5[1].magnitude])
    vsm = np.asarray([umn.magnitude, vmn.magnitude])
    angle_c = np.dot(vshr,vsm)/np.linalg.norm(vshr)/np.linalg.norm(vsm)
    critical_angle = np.arccos(np.clip(angle_c, -1, 1))
    
    return (critical_angle * units('radian')).to('degrees')

In [203]:
def plot_fancy_sounding(stormu, stormv, srh3, srh1, srh_u,
                        srh_v, sup_comp, sigtor_p, 
                        LCL_h , LCL_p, LCL_t, LFC_h,
                        LFC_p, LFC_t, EL_h, EL_p, EL_t, cape, cin, 
                        bottom_eff, top_eff, pwat, h_title, prof, hgt,
                        levc, uc, vc, Tc, Tdc, year, month, day, UTC, station, RM, LM,
                        mean_flow, shr1, shr6, RM_spd, RM_dir, LM_spd, LM_dir, us500, vs500,
                        critical, cape3, mucape, mucin, mu_profile, pres_mu, ebwd, esrh):
    
    #%matplotlib inline
    plt.style.use('bmh')
    #plt.title('21 UTC NARR Sounding 31 May 1985 Wheatland, PA')
    # Change default to be better for skew-T
    fig = plt.figure(figsize=(24, 12))
    #skew = SkewT(fig)
    gs = gridspec.GridSpec(8, 4)
    skew = SkewT(fig, rotation=45, subplot=gs[:, :2])

    # Plot the data using normal plotting functions, in this case using
    # log scaling in Y, as dictated by the typical meteorological plot

    #Add lines for the LCL, LFC, and EL
    if np.isnan(LCL_p) == False:
        skew.ax.axhline(LCL_p, color='gray', linewidth = 3)
        plt.text(LCL_t.magnitude - 30, LCL_p.magnitude - 4, "LCL: "+str(int(LCL_h.magnitude-hgt[0].magnitude))+" m", size = 15)
   
    if np.isnan(LFC_p) == False:
        skew.ax.axhline(LFC_p, color='gray', linewidth = 3)
        plt.text(LFC_t.magnitude - 43, LFC_p.magnitude - 4, "LFC: "+str(int(LFC_h.magnitude-hgt[0].magnitude))+" m", size = 15)
    
    if np.isnan(EL_p) == False :
        skew.ax.axhline(EL_p, color='gray', linewidth = 3)
        plt.text(EL_t.magnitude + 5, EL_p.magnitude - 4, "EL: "+str(int(EL_h.magnitude-hgt[0].magnitude))+" m", size = 15)
   
    if np.isnan(bottom_eff) == False :
        skew.ax.axhline(levc[bottom_eff], xmin = .05, xmax = .15, color='purple', linewidth = 3)
        skew.ax.axhline(levc[top_eff], xmin = .05, xmax = .15, color='purple', linewidth = 3)
   

    skew.plot(levc, Tc, 'r')
    skew.plot(levc, Tdc, 'g')
    skew.plot_barbs(levc, uc, vc)
    
    #Plot the most unstable parcel path
    skew.plot(pres_mu, mu_profile, 'orange', linewidth=4, linestyle = '--')
    #Plot the parcel path
    skew.plot(levc, prof, 'gold', linewidth=4)
   

    #Let's try to fill between the profile and parcel path.
    greater = Tc >= prof
    skew.ax.fill_betweenx(levc, Tc, prof, where=greater, facecolor='blue', alpha=0.4)
    skew.ax.fill_betweenx(levc, Tc, prof, where=~greater, facecolor='red', alpha=0.4)

    skew.ax.set_ylim(1020, 120)
    # Good bounds for aspect ratio
    skew.ax.set_xlim(-30, 40)
    skew.ax.set_xticklabels([-110,-100,-90,-80,-70,-60,-50,-40,-30,-20,-10,0,10,20,30], size = 14)
    skew.ax.set_yticklabels([100,200,300,400,500,600,700,800,900,1000],size = 14)

    plt.title(UTC+" UTC Observed Sounding "+month+"/"+day+"/"+year+" "+station, size = 22)
    l = skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)
    l2 = skew.ax.axvline(-20, color='c', linestyle='--', linewidth=2)

    # Add the relevant special lines
    skew.plot_dry_adiabats()
    skew.plot_moist_adiabats()
    skew.plot_mixing_lines()

    ax1 = fig.add_subplot(gs[:6, 2:])
    h = Hodograph(ax1,component_range=80.)
    h.add_grid(increment=20)
    cl = h.plot_colormapped(uc, vc, hgt, bounds = [0, 1000, 3000, 5000, 10000] * units('meter'),
                 colors = ['magenta', 'red', 'yellow', 'green'], linewidth = 4)
    cb = plt.colorbar(cl, shrink = .8)
    cb.ax.set_yticklabels(['sfc','1,000','3,000','5,000','10,000'])
    cb.set_label('Height AGL (m)')
    h.plot(srh_u, srh_v, color = 'red', linewidth = 2)
    h.plot(us500, vs500, color = 'magenta', linewidth = 2)
    plt.title(h_title, size = 20)
    h.ax.set_xticklabels([-80,-60,-40,-20,0,20,40,60,80],size = 14)
    h.ax.set_yticklabels([-80,-60,-40,-20,0,20,40,60,80],size = 14)

    if stormu:
        h.ax.scatter(stormu.to('knots'), stormv.to('knots'), s = 40, color = 'k')
    h.ax.scatter(RM[0].to('knots'), RM[1].to('knots'), s = 38, color = 'r')
    h.ax.scatter(LM[0].to('knots'), LM[1].to('knots'), s = 38, color = 'b')
    h.ax.scatter(mean_flow[0].to('knots'), mean_flow[1].to('knots'), s = 38, color = 'brown', marker = 'D')
    h.ax.text(RM[0].to('knots'), RM[1].to('knots'), "RM", ha = 'right', color = 'r', size = 18)
    h.ax.text(LM[0].to('knots'), LM[1].to('knots'), "LM", ha = 'right', color = 'b', size = 18)
    #h.ax.text(mean_flow[0], mean_flow[1], "Mean", ha = 'right', color = 'brown', size = 18)
    h.ax.text(LM[0].to('knots'), LM[1].to('knots'), "LM", ha = 'right', color = 'b', size = 18)
    h.ax.text(mean_flow[0].to('knots'), mean_flow[1].to('knots'), "Mean", ha = 'right', color = 'brown', size = 18)

    plt.text(0, -.06, "SBCAPE: "+str(int(cape.magnitude))+" J/kg", size = 15, transform=ax1.transAxes)
    plt.text(.5, -.06, "SBCINH: "+str(int(cin.magnitude))+" J/kg", size = 15, transform=ax1.transAxes)
    plt.text(0, -.095, "SRH 1km: "+str(int(srh1[0].magnitude))+" m^2/s^2", size = 15, transform=ax1.transAxes)
    plt.text(.5, -.095, "SRH 3km: "+str(int(srh3[0].magnitude))+" m^2/s^2", size = 15, transform=ax1.transAxes)
    plt.text(0, -.13, "Sfc-1km Shear: "+str(int(np.round(shr1[2].to(units.knot).magnitude)))+" kt", size = 15, transform=ax1.transAxes)
    plt.text(.5, -.13, "Sfc-6km Shear: "+str(int(np.round(shr6[2].to(units.knot).magnitude)))+" kt", size = 15, transform=ax1.transAxes)
    plt.text(0, -.165, "Supercell: %.2f" %sup_comp, size = 15, transform=ax1.transAxes)
    plt.text(.5, -.165, "Sigtor: %.2f" %sigtor_p, size = 15, transform=ax1.transAxes)
    plt.text(0, -.20, "RM: "+str(int(RM_dir.magnitude))+"/"+str(int(RM_spd.magnitude))+"kt", size = 15, transform=ax1.transAxes)
    plt.text(.5, -.20, "LM: "+str(int(LM_dir.magnitude))+"/"+str(int(LM_spd.magnitude))+"kt", size = 15, transform=ax1.transAxes)
    plt.text(0, -.235, "PW: %.2f" %pwat.to('inches').magnitude +" in", size = 15, transform=ax1.transAxes)
    plt.text(.5, -.235, "3CAPE: "+str(int(cape3.magnitude))+" J/kg", size = 15, transform=ax1.transAxes)
    plt.text(0, -.27, "MUCAPE: "+str(int(mucape.magnitude))+" J/kg", size = 15, transform=ax1.transAxes)
    plt.text(.5, -.27, "MUCIN: "+str(int(mucin.magnitude))+" J/kg", size = 15, transform=ax1.transAxes)
    plt.text(0, -.305, "EBWD: "+str(int(np.round(ebwd.to(units.knot).magnitude)))+" kt", size = 15, transform=ax1.transAxes)
    plt.text(0.5, -.305, "ESRH: "+str(int(esrh.magnitude))+" m^2/s^2", size = 15, transform=ax1.transAxes)

    plt.text(.06, .06, "Critical Angle: "+str(int(critical.magnitude))+" degrees", size = 15, color = 'r', transform=ax1.transAxes)
    


    display(fig)
    #plt.close()
    #plt.title('21 UTC NARR Sounding 31 May 1985 Wheatland, PA')
    #plt.savefig("reallyfancysounding.png")
    

In [204]:

def interactive_sounding(year, month, day, UTC, region, station, speed, direction, temp, dewp, modified, save):
    year = str(year)
    month = str(month)
    day = str(day)
    UTC = str(UTC)
    station = str(station)
    date = datetime(int(year), int(month), int(day), int(UTC))
    ds = get_upper_air_data(date, station, region = region)

    levc = ds.variables['pressure'][:]
    hgt = ds.variables['height'][:]
    Tc = ds.variables['temperature'][:]
    Tdc = ds.variables['dewpoint'][:]
    uc = ds.variables['u_wind'][:]
    vc = ds.variables['v_wind'][:]

    h_title = " "

     #Plot a parcel profile
    prof = metcalc.parcel_profile(levc, Tc[0], Tdc[0]).to('degC')
    
    #Get various wind layers to plot on the hodograph, as well as for sfc-3km cape
    u3 = get_layer(levc, uc, vc, Tc, Tdc, prof, heights = hgt, depth = 3000 * units('meter'))
    u500 = get_layer(levc, uc, vc, heights = hgt, depth = 500 * units('meter'))


    uc3 = u3[1]
    vc3 = u3[2]
    uc500 = u500[1]
    vc500 = u500[2]
    #Get sfc-500m shear vector
    us500 = uc500[[0,-1]]
    vs500 = vc500[[0,-1]]
    #Get Bunkers (2000) storm motions
    RM, LM, mean_flow = bunkers_storm_motion(uc, vc, levc, hgt)
    #Get the speed and direction out of the storm motions
    RM_spd = get_wind_speed(RM[0].to('knots'), RM[1].to('knots'))
    RM_dir = get_wind_dir(RM[0], RM[1])
    LM_spd = get_wind_speed(LM[0].to('knots'), LM[1].to('knots'))
    LM_dir = get_wind_dir(LM[0], LM[1])
    #Make an outline to plot the area used for Sfc-3km SRH
    srh_u = np.asarray([uc3[0].magnitude, RM[0].to('knot').magnitude, uc3[-1].magnitude]) * units.knot
    srh_v = np.asarray([vc3[0].magnitude, RM[1].to('knot').magnitude, vc3[-1].magnitude]) * units.knot

    #Calculate sfc-3 and sfc-1km helicity
    srh3 = storm_relative_helicity(uc, vc, levc, hgt, bottom = hgt[0], top = hgt[0] + 3000 * units('meter'), storm_u = RM[0], storm_v = RM[1])
    srh1 = storm_relative_helicity(uc, vc, levc, hgt, bottom = hgt[0],top = hgt[0] + 1000 * units('meter'), storm_u = RM[0], storm_v = RM[1])
    #srh3 = storm_relative_helicity(uc, vc, levc, hgt, 3000 * units('meter'), storm_u = StormU, storm_v = StormV)

    #Get sfc-6km and sfc-1km shear
    shr6 = bulk_shear(uc, vc, levc, hgt, depth=6000 * units('meter'))
    shr1 = bulk_shear(uc, vc, levc, hgt, depth=1000 * units('meter'))
    shr5 = bulk_shear(uc, vc, levc, hgt, depth=500 * units('meter'))

    #Calculate the critical angle
    critical = critical_angle(uc, vc, levc, hgt, RM[0], RM[1])
    #Calculate LCL, LFC, and EL
    LFC_p, LFC_t = metcalc.lfc(levc, Tc, Tdc)
    LCL_p, LCL_t = metcalc.lcl(levc[0], Tc[0], Tdc[0])
    EL_p, EL_t = metcalc.el(levc, Tc, Tdc)
    #Use log_interp to get heights for these
    LCL_h = log_interp(LCL_p, levc, hgt)
    LFC_h = log_interp(LFC_p, levc, hgt)
    EL_h = log_interp(EL_p, levc, hgt)

    #Feed profile into the cape function to get cape and cin
    cape, cin = cape_cin(levc, Tc, Tdc, prof)
    #temporary fix for LFC bug making cape nan
    if np.isnan(cape):
        cape = 0 * units('J/kg')
    mucape, mucin = mucape_cin(levc, Tc, Tdc)
    if np.isnan(mucape):
        mucape = 0 * units('J/kg')
        mucin = 0 * units('J/kg')
    #Get most unstable profile and EL
    parcel_mu = most_unstable_parcel(levc, Tc, Tdc)
    pres_mu = levc[parcel_mu[-1]:]
    temp_mu = Tc[parcel_mu[-1]:]
    mu_elp, mu_elt = metcalc.el(levc[parcel_mu[-1]:], Tc[parcel_mu[-1]:], Tdc[parcel_mu[-1]:])
    mu_elh = log_interp(mu_elp, levc, hgt)
    mu_profile = parcel_profile(pres_mu, parcel_mu[1], parcel_mu[2]).to('degC')
    #Feed truncated profile into function to get 3km cape and cin
    try:
        cape3, cin3 = cape_cin(u3[0], u3[3], u3[4], u3[5])
    except:
        cape3 = 0 * units('J/kg')
        
    eff_layer = get_layer(levc, Tc, Tdc, depth = levc[0] - 400 * units('hPa'))
    bottom_eff, top_eff = effective_layer(levc, Tc, Tdc)
    
    # Calculate effective bulk shear
    if (not np.isnan(bottom_eff)) and (bottom_eff != top_eff) and (not np.isnan(mu_elh)):
        _, _, ebwd = bulk_shear(uc, vc, levc, hgt, bottom=hgt[bottom_eff], depth=((.5 * (mu_elh-hgt[0]))+hgt[0])-hgt[bottom_eff])
    else:
        ebwd = 0 * units('m/s')
    if (not np.isnan(bottom_eff)) and (bottom_eff != top_eff):
        esrh, _, _ = storm_relative_helicity(uc, vc, levc, hgt, bottom = hgt[bottom_eff], top = hgt[top_eff], storm_u = RM[0],
                                       storm_v = RM[1])
    else:
        esrh = 0 * units('m^2/s^2')
    
    
    # calculate supercell comp and sigtor
    sup_comp = supercell_comp(np.asarray([mucape.magnitude]) * units('J/kg'), np.asarray([srh3[0].magnitude]) * units('m^2/s^2'),
                              np.asarray([ebwd.magnitude]) * units('m/s'))

    sigtor_p = sigtor(np.asarray([cape.magnitude]) * units('J/kg'), np.asarray([LCL_h.magnitude-hgt[0].magnitude]) * units('meter'),
                      np.asarray([srh1[0].magnitude]) * units('m^2/s^2'), np.asarray([shr6[2].magnitude]) * units('m/s'))

    pwat = precipitable_water(Tdc, levc).to('inches')
    

    sfct = Tc[0]
    sfcd = Tdc[0]
    #print(RM_dir)
    
    if modified == True:
        #Check to see which parameters have been modified.
        if speed == 0.001:
            speed = RM_spd.magnitude
        if direction == 0.001:
            direction = RM_dir.magnitude
        if temp == 0.001:
            temp = sfct.magnitude
        if dewp == 0.001:
            dewp = sfcd.magnitude
        if speed != RM_spd.magnitude or direction != RM_dir.magnitude or temp != sfct.magnitude or dewp != sfcd.magnitude:
            h_title = "Selected Storm Motion: "+str(int(direction))+"/"+str(int(speed))+"kt"
            stormu, stormv = get_wind_components(speed, direction * units.deg) * units('knot')
            #temporary fix making the hgts agl until storm_relative_helicity is fixed.
            srh3 = storm_relative_helicity(uc, vc, levc, hgt, bottom = hgt[0], top = hgt[0] + 3000 * units('meter'), storm_u = stormu, storm_v = stormv)
            srh1 = storm_relative_helicity(uc, vc, levc, hgt, bottom = hgt[0], top = hgt[0] + 1000 * units('meter'), storm_u = stormu, storm_v = stormv)
            srh_u = np.asarray([uc3[0].magnitude, stormu.magnitude, uc3[-1].magnitude]) * units.knot
            srh_v = np.asarray([vc3[0].magnitude, stormv.magnitude, vc3[-1].magnitude]) * units.knot
            critical = critical_angle(uc, vc, levc, hgt, stormu, stormv)
            
            if temp != sfct.magnitude or dewp != sfcd.magnitude:
                Tc[0] = temp * units.degC
                Tdc[0] = dewp * units.degC
                #Calculate LCL, LFC, and EL
                LFC_p, LFC_t = metcalc.lfc(levc, Tc, Tdc)
                LCL_p, LCL_t = metcalc.lcl(levc[0], Tc[0], Tdc[0])
                EL_p, EL_t = metcalc.el(levc, Tc, Tdc)
                #Use log_interp to get heights for these
                LCL_h = log_interp(LCL_p, levc, hgt)
                LFC_h = log_interp(LFC_p, levc, hgt)
                EL_h = log_interp(EL_p, levc, hgt)
                pwat = precipitable_water(Tdc, levc)
                eff_layer = get_layer(levc, Tc, Tdc, depth = levc[0] - 400 * units('hPa'))
                bottom_eff, top_eff = effective_layer(levc, Tc, Tdc)

                #Plot a parcel profile
                prof = metcalc.parcel_profile(levc, Tc[0], Tdc[0]).to('degC')
                u3 = get_layer(levc, uc, vc, Tc, Tdc, prof, heights = hgt, depth = 3000 * units('meter'))

                #Feed it into the cape function to get cape and cin
                cape, cin = cape_cin(levc, Tc, Tdc, prof)
                cape3, cin3 = cape_cin(u3[0], u3[3], u3[4], u3[5])
                mucape, mucin = mucape_cin(levc, Tc, Tdc)
                #Get most unstable profile and EL
                parcel_mu = most_unstable_parcel(levc, Tc, Tdc)
                pres_mu = levc[parcel_mu[-1]:]
                temp_mu = Tc[parcel_mu[-1]:]
                mu_elp, mu_elt = metcalc.el(levc[parcel_mu[-1]:], Tc[parcel_mu[-1]:], Tdc[parcel_mu[-1]:])
                mu_elh = log_interp(mu_elp, levc, hgt)
                mu_profile = parcel_profile(pres_mu, parcel_mu[1], parcel_mu[2]).to('degC')
                # Calculate effective bulk shear
                if (not np.isnan(bottom_eff)) and (bottom_eff != top_eff) and (not np.isnan(mu_elh)):
                    _, _, ebwd = bulk_shear(uc, vc, levc, hgt, bottom=hgt[bottom_eff], depth=((.5 * (mu_elh-hgt[0]))+hgt[0])-hgt[bottom_eff])
                else:
                    ebwd = 0 * units('m/s')

            if (not np.isnan(bottom_eff)) and (bottom_eff != top_eff):
                esrh, _, _ = storm_relative_helicity(uc, vc, levc, hgt, bottom = hgt[bottom_eff], top = hgt[top_eff], storm_u = RM[0],
                                               storm_v = RM[1])
            else:
                esrh = 0 * units('m^2/s^2')
            #calculate supercell comp and sigtor
            sup_comp = supercell_comp(np.asarray([mucape.magnitude]) * units('J/kg'), 
                                      np.asarray([srh3[0].magnitude]) * units('m^2/s^2'), 
                                      np.asarray([shr6[2].magnitude]) * units('m/s'))

            sigtor_p = sigtor(np.asarray([cape.magnitude]) * units('J/kg'), 
                              np.asarray([LCL_h.magnitude-hgt[0].magnitude]) * units('meter'), 
                              np.asarray([srh1[0].magnitude]) * units('m^2/s^2'), np.asarray([shr6[2].magnitude]) * units('m/s'))

            #print("Well there's your problem") 

            plot_fancy_sounding(stormu = stormu, stormv = stormv, srh3 = srh3, srh1 = srh1, 
                            srh_u = srh_u, srh_v = srh_v, sup_comp = sup_comp, sigtor_p = sigtor_p, 
                            LCL_h = LCL_h, LCL_p = LCL_p, LCL_t = LCL_t, LFC_h = LFC_h, LFC_p = LFC_p, LFC_t = LFC_t,
                            EL_h = EL_h, EL_p = EL_p, EL_t = EL_t,
                            cape = cape, cin = cin, bottom_eff = bottom_eff, top_eff =top_eff,
                            pwat = pwat, h_title = h_title, prof = prof, hgt = hgt, levc = levc, uc = uc, vc = vc, Tc = Tc,
                            Tdc = Tdc, year = year, month = month, day = day, UTC = UTC, station = station, RM = RM, LM = LM, 
                            mean_flow = mean_flow, shr1 = shr1, shr6 = shr6,
                            RM_dir = RM_dir, RM_spd = RM_spd, LM_dir = LM_dir, LM_spd = LM_spd, us500 = us500, vs500 = vs500,
                            critical = critical, cape3 = cape3, mucape = mucape, mucin = mucin, mu_profile = mu_profile, pres_mu = pres_mu,
                            ebwd = ebwd, esrh = esrh)
    

    else:
        Tc[0] = sfct
        Tdc[0] = sfcd
        plot_fancy_sounding(stormu = RM[0], stormv = RM[1], srh3 = srh3, srh1 = srh1, 
                            srh_u = srh_u, srh_v = srh_v, sup_comp = sup_comp, sigtor_p = sigtor_p, 
                            LCL_h = LCL_h, LCL_p = LCL_p, LCL_t = LCL_t, LFC_h = LFC_h, LFC_p = LFC_p, LFC_t = LFC_t,
                            EL_h = EL_h, EL_p = EL_p, EL_t = EL_t,
                            cape = cape, cin = cin, bottom_eff = bottom_eff, top_eff =top_eff,
                            pwat = pwat, h_title = h_title, prof = prof, hgt = hgt, levc = levc, uc = uc, vc = vc, Tc = Tc,
                            Tdc = Tdc, year = year, month = month, day = day, UTC = UTC, station = station, RM = RM, LM = LM, 
                            mean_flow = mean_flow, shr1 = shr1, shr6 = shr6,
                            RM_dir = RM_dir, RM_spd = RM_spd, LM_dir = LM_dir, LM_spd = LM_spd, us500 = us500, vs500 = vs500,
                            critical = critical, cape3 = cape3, mucape = mucape, mucin = mucin, mu_profile = mu_profile, pres_mu = pres_mu,
                            ebwd = ebwd, esrh = esrh)
    if save == True:
        plt.savefig(UTC+"UTCObservedSounding"+month+"-"+day+"-"+year+station+".png")
    
    plt.close()

In [205]:
region_list = {'North America': 'naconf',
                'South America': 'samer',
                'South Pacific': 'pac',
                'New Zealand': 'nz',
                'Antarctica': 'ant',
                'Arctic': 'np',
                'Europe': 'europe',
                'Africa': 'africa',
                'Southeast Asia': 'seasia',
                'Mideast' : 'mideast'}

In [206]:

interact_manual(interactive_sounding, year = Textarea(description = 'Year', value = '2016'),
                month = Textarea(description = 'Month', value = '05'),
                day = Textarea(description = 'Day', value = '23'),
                UTC = Textarea(description = 'UTC', value = '00'),
                region = Dropdown(options=region_list,value='naconf',description='Region:'),
                station = Textarea(description = 'Site', value = 'DDC'),
                speed=FloatSlider(min=0, max=100, value = 0.001, step=1, description = "Storm Speed"),
                direction = FloatSlider(min=0, max=360, value = 0.001, step=1, description = "Storm Direction"),
                temp = FloatSlider(min = -30, max = 45, value=0.001,description='Sfc Temp:'),
                dewp = FloatSlider(min = -40, max = 40, value=0.001,description='Sfc Td:'),
                modified = Checkbox(description = "Modified?"), save = Checkbox(description = "Save Sounding?"))

<function __main__.interactive_sounding>

In [207]:
7/18/2017 00Z ILN

SyntaxError: invalid syntax (<ipython-input-207-8f5f6f615044>, line 1)

In [ ]:
7/19/2017 12Z 33345 (Kiev, Ukraine) LFC shows up below LCL in deep mixed layer 

In [ ]:
7/19/2017 31088 (Russia, Arctic region) LFC just below LCL

In [ ]:
2/2/2011 00 ILX May or may not be an error, but LFC=LCL when whole trace might be colder than environment.

In [ ]:
4/15/2011 00 ILX Slightly superadiabatic PBL meets a substantial cap just above it. Parcel path slightly warmer than
environment through PBL and colder therafter. LFC and EL should not exist, but LFC is plotted at the LCL and EL plotted
below both.

In [ ]:
5/15/2011 00 ILX Likely error with MUCAPE, shows none when there should likely be at least 100J/kg

In [ ]:
3/8/2008 00 ILN Really weird sounding with a very shallow unstable layer where it appears everything is actually correct.

5/10/2010 OUN 00Z Metpy doesn't play well with the superadiabatic near-sfc layer here. 